In [2]:
pwd

'/root/eks-ml-pipeline'

In [3]:
!pip uninstall -y msspackages

Keyring is skipped due to an exception: 'keyring.backends'
Found existing installation: msspackages 0.0.7
Uninstalling msspackages-0.0.7:
  Successfully uninstalled msspackages-0.0.7


In [4]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl


In [5]:
from msspackages import setup_runner
setup_runner()

b"Get:1 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nHit:2 http://deb.debian.org/debian buster InRelease\nHit:3 http://deb.debian.org/debian buster-updates InRelease\nFetched 34.8 kB in 0s (105 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n52 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nsudo is already the newest version (1.8.27-1+deb10u4).\n0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\ndefault-jre is already the newest version (2:1.11-71).\n0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\npython3 is already the newest version (3.7.3-1).\n0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded

In [6]:
!python3 --version

Python 3.7.10


In [7]:
!python3 -m pip install -r requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'


<h1>Node, Pod & Container Training & Testing Data - Autoencoder<h1>

<h1>Inference Data Builder<h1>

In [ ]:
from eks_ml_pipeline import inference_data_builder

In [ ]:
inference_data_builder("2022","9","11","12","Node","128gb")

In [ ]:
inference_data_builder("2022","7","10","20","Pod","128gb")

In [ ]:
inference_data_builder("2022","8","20","9","Container","128gb")

<h1> sample df and tensor write functions<h1>

In [ ]:
from eks_ml_pipeline import write_tensor, awswrangler_pandas_dataframe_to_s3

In [ ]:
awswrangler_pandas_dataframe_to_s3(pod_training_data,  "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [ ]:
write_tensor(pod_training_tensor, "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [ ]:
!pip install fastparquet

In [ ]:
time_steps = 20
batch_size = 32
train_n_samples = batch_size * 1000
test_n_samples = round((batch_size*1000*0.2) / 0.8)

In [ ]:
features = ['pod_cpu_utilization','pod_memory_utilization','pod_network_total_bytes']
scaled_features = ['scaled_pod_cpu_utilization','scaled_pod_memory_utilization','scaled_pod_network_total_bytes']

In [ ]:
!pip install fastparquet
import pandas as pd
pod_training_df = pd.read_parquet('training_pod_2022_9_9.parquet', engine='fastparquet')
pod_testing_df = pd.read_parquet('testing_pod_2022_9_9.parquet', engine='fastparquet')


In [ ]:
pod_training_df.shape

In [ ]:
pod_testing_df.shape

In [ ]:
pod_training_df['freq'] = pod_training_df.groupby('pod_id')['pod_id'].transform('count')

In [ ]:
filtered_pod_df = pod_training_df[pod_training_df["freq"] > 20]

In [ ]:
selected_pod_list = filtered_pod_df['pod_id'].sample(train_n_samples).to_list()

In [ ]:
len(selected_pod_list)

In [ ]:
from sklearn.preprocessing import StandardScaler
import random

def pod_tensor_builder(pod_id):
    
    features = ['pod_cpu_utilization','pod_memory_utilization','pod_network_total_bytes']
    scaled_features = ['scaled_pod_cpu_utilization','scaled_pod_memory_utilization','scaled_pod_network_total_bytes']

    ##pick random df, and normalize
    pod_fe_df = filtered_pod_df.loc[(filtered_pod_df["pod_id"] == pod_id)]
    pod_fe_df = pod_fe_df.sort_values(by='Timestamp').reset_index(drop=True)
    pod_fe_df_len = len(pod_fe_df)


    #tensor builder
    start = random.choice(range(pod_fe_df_len-20))
    pod_fe_df = pod_fe_df[start:start+20]

    #scaler transformations
    scaler = StandardScaler()
    pod_fe_df[scaled_features] = scaler.fit_transform(pod_fe_df[features])

    return pod_fe_df

In [ ]:
%%time
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
results = multiprocessing.Pool(num_cores).map(pod_tensor_builder, selected_pod_list)

In [ ]:
1+1

In [ ]:
len(results)

In [ ]:
pod_training_df = pd.concat(results)

In [ ]:
pod_training_df.shape

In [ ]:
import numpy as np
pod_training_tensor = np.array(list(map(lambda x: x.to_numpy(), results)))

In [ ]:
pod_training_tensor.shape

In [8]:
from eks_ml_pipeline import write_tensor, awswrangler_pandas_dataframe_to_s3,read_tensor

In [ ]:
write_tensor(pod_training_tensor, "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [ ]:
awswrangler_pandas_dataframe_to_s3(pod_training_df,  "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [9]:
data_train = read_tensor('dish-5g.core.pd.g.dp.eks.logs.e','pod_autoencoder_ad','v0.0.2','training_2022_9_9_scaled_features')

reading tensor from: dish-5g.core.pd.g.dp.eks.logs.e/pod_autoencoder_ad/v0.0.2/data/tensors/training_2022_9_9_scaled_features.npy


In [10]:
data_train

array([[[-0.8654696713508648, -0.5377736036854034, -0.10870967870564109],
        [1.13370640875588, -0.4540767893131924, 1.2645071314684502],
        [-0.8182498826202578, -0.37641995123588184, -0.7394691467412315],
        ...,
        [1.2266475317231291, -0.2633861091455646, 0.6654711894530544],
        [1.6554316185532278, -0.27115179295325814, 1.6100782415863173],
        [-0.9184546057192291, -0.34104294722287515, -1.1168237485891575]],

       [[0.6823707867301055, -0.4365022328229814, -0.6166181582167075],
        [-1.6695822452982017, -0.5798788786407524, -0.2232797080982047],
        [1.1099615822071, -0.13381820276328177, 2.08332773586054],
        ...,
        [0.6544045348009389, 0.20072730414486853, -0.6725074488083416],
        [-0.9290573256426782, 0.22462341178114567, -0.5448082166832063],
        [0.3831074455704446, -1.6153768762135188, -0.37357055982952975]],

       [[0.0, 0.0, -1.869060920713966],
        [0.0, 0.0, -1.3722687235939566],
        [0.0, 0.0, -1.521